In [30]:
import numpy
import subprocess
import pathlib

filmod_name = 'mode.vasp'
fileig_name = 'Mp.eig'
filscf_name = 'scf.in'
q = [0.5, 0.5, 0.0]
facs = [None]
phase = numpy.pi/2
amp = 1

filscf = open(filscf_name, 'r')
au_to_angst = 0.529177249

#get ntyp, nat and alat
for line in filscf:
    #get nat
    if 'nat' in line:
        nat = int(line.split()[2])
    #get ntyp
    elif 'ntyp' in line:
        ntyp = int(line.split()[2])
    #get lattice vectors and calculate reciprocal lattice vectors
    elif 'CELL_PARAMETERS' in line:
        lat = numpy.empty(shape=(3,3), dtype=float)
        for i in range(0, 3):
            lv = filscf.readline().split()
            for j in range(0, 3):
                lat[i, j] = float(lv[j])
        lat_inv = 2*numpy.pi * numpy.linalg.inv(lat)
    #get atomic specifications
    elif 'ATOMIC_SPECIES' in line:
        atname = numpy.empty(ntyp, dtype=str)
        atmass = numpy.empty(ntyp, dtype=float)
        for i in range(0, ntyp):
            line = filscf.readline().split()
            atname[i] = line[0]
            atmass[i] = float(line[1])
    #get atomic positions and atom type 
    elif 'ATOMIC_POSITIONS' in line:
        at = numpy.empty(shape=(nat,4), dtype=float)
        for i in range(0, nat):
            pos = filscf.readline().split()
            
            at[i, 0] = numpy.where(atname == pos[0])[0][0]
            for j in range(1, 4):
                at[i, j] = float(pos[j])    
filscf.close()

fileig = open(fileig_name, 'r')
for i in range(0, 2):
    fileig.readline()
    
#get q-vector    
if q == [None]:
    q = numpy.empty(3, dtype=float)
    vec = fileig.readline().split()
    for i in range(0, 3):
        q[i] = float(vec[i + 2])

else:
    q = numpy.array(q)
    fileig.readline()
    
if facs == [None]:    
    facs = numpy.empty(3, dtype=int)
    for i, xq in enumerate(q):
        if xq == 0:
            facs[i] = 1
            continue
        facs[i] = 1/xq
else:
    facs = numpy.array(facs)

lat_mod = numpy.copy(lat)
for i, fac in enumerate(facs):
    lat_mod[i] *= fac
    
fileig.readline()   

#get modulations
nmod = 3*nat
mods = numpy.empty(shape=(nmod, nat, 3), dtype=complex)
for i in range(0, nmod):
    fileig.readline()
    for j in range(0, nat):
        vec = fileig.readline().split()
        for k in range(0, 3):
            mods[i,j,k] = complex(float(vec[k*2+1]), float(vec[k*2+2]))

#generate supercell with modulation
for imod in range(0, nmod):
    nat_mod = nat
    at_mod = numpy.empty(shape=(0,4), dtype=float)
        
    for i in range(0, facs[0]):
        for j in range(0, facs[1]):
            for k in range(0, facs[2]):
                for l, iat in enumerate(at):
                    pos = numpy.copy(iat)

                    pos[1] += i
                    pos[2] += j
                    pos[3] += k
                    
                    pos[1] /= facs[0]
                    pos[2] /= facs[1]
                    pos[3] /= facs[2]
                    
                    pos_abs = numpy.dot(lat_mod, pos[1:4])
                    q_abs = numpy.dot(lat_inv, q)
                    wave = numpy.exp(1j*numpy.dot(q_abs,pos_abs)+1j*phase)
                    disp = mods[imod,l]
                    pos[1:4] += amp*(disp*wave).astype(float) / numpy.sqrt(atmass[int(pos[0])])
                
                    at_mod = numpy.vstack((at_mod, pos))
        #sort atoms
    sortby = at_mod[:,0].argsort()
    at_mod = at_mod[sortby[::1]]
       
    #write vasp file
    filmod = open(filmod_name + str(imod), 'w')
    filmod.write('\n1.0\n')
    for lv in lat_mod:
        for num in lv:
            filmod.write('\t' + str(num) + '\t')
        filmod.write('\n')
        
    nat_typ = numpy.bincount(at_mod[:,0].astype(int))
    for iname in atname:
        filmod.write(iname + '\t')
    filmod.write('\n')
    for i, n in enumerate(nat_typ):
        filmod.write(str(n) + '\t')
    filmod.write('\nDirect\n')
    for iat in at_mod:
        for i, num in enumerate(iat):
            if i == 0:
                continue
            filmod.write(str(num) + '\t')
        filmod.write('\n')
    filmod.close()



/usr/lib/python3/dist-packages/ipykernel_launcher.py:114: ComplexWarning: Casting complex values to real discards the imaginary part
